In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Set up "**Console Access Library**"

## Imports

In [ ]:
import errno
import json
from pathlib import Path

import jsonschema
from console_access_library.client import Client
from console_access_library.common.config import Config

## Load Configurations

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file.
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file.
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration.
jsonschema.validate(json_load, json_schema)

## Create API Client

In [ ]:
# Instantiate Console Access Library Config.
print("Create access lib client...")

try:
    config_obj = Config(**json_load)
except Exception as e:
    # EXCEPTION
    raise e

# Instantiate Console Access Library Client.
client_obj = Client(config_obj)

# Check if settings are valid.
print("Checking if settings are valid...")
device_management_obj = client_obj.get_device_management()

try:
    response = device_management_obj.get_devices()
except Exception as e:
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
%store client_obj
print("Console access lib client setup complete.")